In [2]:
!pip install fitz


     ---------------------------------------- 95.6/95.6 kB 5.7 MB/s eta 0:00:00
     ---------------------------------------- 3.2/3.2 MB 14.6 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 17.5 MB/s eta 0:00:00
     ------------------------------------- 564.9/564.9 kB 11.8 MB/s eta 0:00:00
     ---------------------------------------- 43.2/43.2 kB ? eta 0:00:00
     ---------------------------------------- 75.6/75.6 kB 4.4 MB/s eta 0:00:00
     ---------------------------------------- 5.0/5.0 MB 17.9 MB/s eta 0:00:00
     -------------------------------------- 421.5/421.5 kB 6.5 MB/s eta 0:00:00
     ------------------------------------- 528.1/528.1 kB 11.0 MB/s eta 0:00:00
     ---------------------------------------- 41.7/41.7 kB 2.0 MB/s eta 0:00:00


In [4]:
!pip install pymupdf

     ---------------------------------------- 16.5/16.5 MB 7.5 MB/s eta 0:00:00


In [5]:
import fitz  # PyMuPDF for PDF text extraction
import re
import nltk
from nltk.tokenize import sent_tokenize

In [6]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to C:\Users\Sumiran
[nltk_data]     Grover\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF file and removes extra spaces and line breaks."""
    doc = fitz.open(pdf_path)
    text = ""

    for page in doc:
        text += page.get_text("text") + "\n"

    # Remove multiple spaces and newlines
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [8]:
def clean_text(text):
    """Preprocesses extracted text by removing references, citations, and special characters."""
    # Remove in-text citations like [1], (Smith et al., 2020)
    text = re.sub(r'\[\d+\]', '', text)
    text = re.sub(r'\(.*?\d{4}.*?\)', '', text)

    # Remove special characters except basic punctuation
    text = re.sub(r'[^a-zA-Z0-9.,!?;:()\s]', '', text)
    
    return text

In [9]:
def split_into_sections(text):
    """Splits research paper text into logical sections using common headers."""
    sections = {
        "Title": "",
        "Abstract": "",
        "Introduction": "",
        "Methods": "",
        "Results": "",
        "Discussion": "",
        "Conclusion": "",
    }

    sentences = sent_tokenize(text)  # Tokenize into sentences

    current_section = "Title"
    for sentence in sentences:
        # Detect section headers
        if "abstract" in sentence.lower():
            current_section = "Abstract"
        elif "introduction" in sentence.lower():
            current_section = "Introduction"
        elif "method" in sentence.lower():
            current_section = "Methods"
        elif "result" in sentence.lower():
            current_section = "Results"
        elif "discussion" in sentence.lower():
            current_section = "Discussion"
        elif "conclusion" in sentence.lower():
            current_section = "Conclusion"

        # Append to the appropriate section
        sections[current_section] += sentence + " "

    return sections


In [11]:
pdf_path = "A_Comprehensive_Review_of_Unimodal_and_Multimodal_Fingerprint_Biometric_Authentication_Systems_Fusion_Attacks_and_Template_Protection.pdf"  # Change to your file path
raw_text = extract_text_from_pdf(pdf_path)
cleaned_text = clean_text(raw_text)
sections = split_into_sections(cleaned_text)


In [12]:
for section, content in sections.items():
    print(f"\n=== {section} ===\n{content[:500]}...\n") 


=== Title ===
Received 7 March 2024, accepted 25 April 2024, date of publication 30 April 2024, date of current version 13 May 2024. Digital Object Identifier 10.1109ACCESS.2024.3395417 A Comprehensive Review of Unimodal and Multimodal Fingerprint Biometric Authentication Systems: Fusion, Attacks, and Template Protection U. SUMALATHA 1, K. KRISHNA PRAKASHA 1, , Manipal Academy of Higher Education , Manipal Academy of Higher Education  and Srikanth Prabhu  Universality: Anyone must be able to use the applica t...


=== Abstract ===
...


=== Introduction ===
A. K. Jain, A. Ross, and S. Prabhakar, An introduction to biometric recognition, IEEE Trans. Circuits Syst. Video Technol., vol. 14, no. 1, pp. 420, Jan. 2004. A. K. Jain, A. Ross, and K. Nandakumar, Introduction to Biometrics. Cham, Switzerland: Springer, 2016. K. ElMaleh and W. ElHajj, Voice biometrics: Security, forensics, and healthcare, J. Med. Syst., vol. 43, no. 9, p. 306, 2019. K. Banerjee, J. P. Singh, and R. Kaur, Human r

In [13]:
import spacy
from transformers import pipeline

In [16]:
!pip install --upgrade transformers
!pip install tensorflow
!pip install keras

     ---------------------------------------- 9.7/9.7 MB 9.7 MB/s eta 0:00:00
     -------------------------------------- 303.8/303.8 kB 6.2 MB/s eta 0:00:00
     ---------------------------------------- 2.4/2.4 MB 13.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.11.4
    Uninstalling tokenizers-0.11.4:
      Successfully uninstalled tokenizers-0.11.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.24.0
    Uninstalling transformers-4.24.0:
      Successfully uninstalled transformers-4.24.0


In [18]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer

class ResearchSummarizer:
    def __init__(self):
        self.nlp = spacy.load('en_core_web_sm')
    
    def extract_summary(self, text, max_sentences=3):
        """Generate extractive summary using TF-IDF"""
        sentences = [sent.text for sent in self.nlp(text).sents]
        
        vectorizer = TfidfVectorizer(stop_words='english')
        tfidf_matrix = vectorizer.fit_transform(sentences)
        
        sentence_scores = tfidf_matrix.sum(axis=1)
        top_sentence_indices = sorted(
            range(len(sentence_scores)), 
            key=lambda i: sentence_scores[i], 
            reverse=True
        )[:max_sentences]
        
        return ' '.join([sentences[i] for i in sorted(top_sentence_indices)])
    
    def extract_keywords(self, text, top_k=5):
        """Extract top keywords using TF-IDF"""
        vectorizer = TfidfVectorizer(stop_words='english')
        tfidf_matrix = vectorizer.fit_transform([text])
        feature_names = vectorizer.get_feature_names_out()
        
        sorted_indices = tfidf_matrix.toarray()[0].argsort()[::-1]
        return [feature_names[i] for i in sorted_indices[:top_k]]

In [19]:
def evaluate_summarizer(sections):
    summarizer = ResearchSummarizer()
    
    results = {}
    for section, content in sections.items():
        summary = summarizer.extract_summary(content)
        keywords = summarizer.extract_keywords(content)
        
        results[section] = {
            'summary_length': len(summary.split()),
            'summary': summary,
            'keywords': keywords
        }
    
    # Simple quality metrics
    print("Summary Quality Metrics:")
    for section, data in results.items():
        print(f"{section}:")
        print(f"  Summary Length: {data['summary_length']} words")
        print(f"  Keywords: {data['keywords']}")
        print(f"  Summary: {data['summary']}\n")
    
    return results

# Run evaluation
summary_results = evaluate_summarizer(sections)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [20]:
import fitz
import re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def clean_text(text):
    text = re.sub(r'\[\d+\]', '', text)
    text = re.sub(r'\(.*?\d{4}.*?\)', '', text)
    text = re.sub(r'[^a-zA-Z0-9.,!?;:()\s]', '', text)
    return text

def split_into_sections(text):
    sections = {
        "Title": "",
        "Abstract": "",
        "Introduction": "",
        "Methods": "",
        "Results": "",
        "Discussion": "",
        "Conclusion": "",
    }
    sentences = sent_tokenize(text)
    current_section = "Title"
    for sentence in sentences:
        if "abstract" in sentence.lower():
            current_section = "Abstract"
        elif "introduction" in sentence.lower():
            current_section = "Introduction"
        elif "method" in sentence.lower():
            current_section = "Methods"
        elif "result" in sentence.lower():
            current_section = "Results"
        elif "discussion" in sentence.lower():
            current_section = "Discussion"
        elif "conclusion" in sentence.lower():
            current_section = "Conclusion"
        sections[current_section] += sentence + " "
    return sections

class ResearchSummarizer:
    def __init__(self):
        self.nlp = spacy.load('en_core_web_sm')
    
    def extract_summary(self, text, max_sentences=3):
        sentences = [sent.text for sent in self.nlp(text).sents]
        
        vectorizer = TfidfVectorizer(stop_words='english')
        tfidf_matrix = vectorizer.fit_transform(sentences)
        
        sentence_scores = tfidf_matrix.sum(axis=1)
        top_sentence_indices = sorted(
            range(len(sentence_scores)), 
            key=lambda i: sentence_scores[i], 
            reverse=True
        )[:max_sentences]
        
        return ' '.join([sentences[i] for i in sorted(top_sentence_indices)])
    
    def extract_keywords(self, text, top_k=5):
        vectorizer = TfidfVectorizer(stop_words='english')
        tfidf_matrix = vectorizer.fit_transform([text])
        feature_names = vectorizer.get_feature_names_out()
        
        sorted_indices = tfidf_matrix.toarray()[0].argsort()[::-1]
        return [feature_names[i] for i in sorted_indices[:top_k]]

def analyze_paper(pdf_path):
    # Extract and process text
    raw_text = extract_text_from_pdf(pdf_path)
    cleaned_text = clean_text(raw_text)
    sections = split_into_sections(cleaned_text)
    
    # Initialize summarizer
    summarizer = ResearchSummarizer()
    
    # Generate insights for each section
    insights = {}
    for section, content in sections.items():
        if content.strip():
            insights[section] = {
                'summary': summarizer.extract_summary(content),
                'keywords': summarizer.extract_keywords(content)
            }
    
    # Print results
    for section, data in insights.items():
        print(f"\n{section.upper()}:")
        print("Summary:", data['summary'])
        print("Keywords:", data['keywords'])
    
    return insights

# Usage
pdf_path = "A_Comprehensive_Review_of_Unimodal_and_Multimodal_Fingerprint_Biometric_Authentication_Systems_Fusion_Attacks_and_Template_Protection.pdf"
paper_insights = analyze_paper(pdf_path)


TITLE:
Summary: Digital Object Identifier 10.1109ACCESS.2024.3395417 A Comprehensive Review of Unimodal and Multimodal Fingerprint Biometric Authentication Systems: Fusion, Attacks, and Template Protection U. SUMALATHA 1, K. KRISHNA PRAKASHA 1, , Manipal Academy of Higher Education , Manipal Academy of Higher Education  and Srikanth Prabhu  Universality: Anyone must be able to use the applica tion using biometric features. Performance: The overall accuracy should be high, with low False Acceptance Rate  illumination due to its sensitivity to light . Similarly, palmprints boast reliability and distinctiveness through ridge patterns, principal lines, minutiae details, delta points, and complex textures , while knuckleprints offer discriminating features captured by various sensors .
Keywords: ['biometric', '2024', 'accuracy', 'fingerprint', 'traits']

INTRODUCTION:
Summary: A. B. A. Hassanat, V. B. S. Prasath, M. Alkasassbeh, A. S. Tarawneh, and A. J. Alshamailh, Magnetic energybased fe

In [21]:
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM,  # For abstractive summarization
)

In [22]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
class HybridSummarizer:
    def __init__(self, 
                 extractive_model='en_core_web_sm', 
                 abstractive_model='facebook/bart-large-cnn'):
        # Extractive model setup
        self.nlp = spacy.load(extractive_model)
        
        # Abstractive model setup
        self.tokenizer = AutoTokenizer.from_pretrained(abstractive_model)
        self.abstractive_model = AutoModelForSeq2SeqLM.from_pretrained(abstractive_model)
        
        # Section summarization strategy
        self.summarization_strategy = {
            'Methods': self.extractive_summary,
            'Results': self.extractive_summary,
            'Introduction': self.abstractive_summary,
            'Discussion': self.abstractive_summary,
            'Conclusion': self.abstractive_summary
        }

In [24]:
def extractive_summary(self, text, max_sentences=3):
        """Generate extractive summary using TF-IDF"""
        sentences = [sent.text for sent in self.nlp(text).sents]
        
        vectorizer = TfidfVectorizer(stop_words='english')
        tfidf_matrix = vectorizer.fit_transform(sentences)
        
        sentence_scores = tfidf_matrix.sum(axis=1)
        top_sentence_indices = sorted(
            range(len(sentence_scores)), 
            key=lambda i: sentence_scores[i], 
            reverse=True
        )[:max_sentences]
        
        return ' '.join([sentences[i] for i in sorted(top_sentence_indices)])

In [25]:
def abstractive_summary(self, text, max_length=150, min_length=50):
        """Generate abstractive summary using BART"""
        inputs = self.tokenizer(
            text, 
            max_length=1024, 
            return_tensors='pt', 
            truncation=True
        )
        
        summary_ids = self.abstractive_model.generate(
            inputs['input_ids'], 
            num_beams=4, 
            max_length=max_length, 
            min_length=min_length,
            early_stopping=True
        )
        
        return self.tokenizer.decode(
            summary_ids[0], 
            skip_special_tokens=True
        )
    

In [26]:
 def summarize_sections(self, sections):
        """
        Generate summaries for different sections using 
        appropriate summarization strategy
        """
        summaries = {}
        for section, content in sections.items():
            if content.strip():
                # Select summarization method based on section
                summarizer = self.summarization_strategy.get(
                    section, 
                    self.extractive_summary  # Default fallback
                )
                summaries[section] = summarizer(content)
        
        return summaries

In [27]:
def process_research_paper(pdf_path):
    # Existing text extraction and cleaning functions
    raw_text = extract_text_from_pdf(pdf_path)
    cleaned_text = clean_text(raw_text)
    sections = split_into_sections(cleaned_text)
    
    # Initialize hybrid summarizer
    summarizer = HybridSummarizer()
    
    # Generate section summaries
    section_summaries = summarizer.summarize_sections(sections)
    
    # Print or further process summaries
    for section, summary in section_summaries.items():
        print(f"{section} Summary:")
        print(summary)
        print("-" * 50)
    
    return section_summaries


In [28]:
pdf_path = "A_Comprehensive_Review_of_Unimodal_and_Multimodal_Fingerprint_Biometric_Authentication_Systems_Fusion_Attacks_and_Template_Protection.pdf"
summaries = process_research_paper(pdf_path)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

C:\Users\Sumiran Grover\anaconda3\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sumiran Grover\.cache\huggingface\hub\models--facebook--bart-large-cnn. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Sumiran Grover\anaconda3\lib\site-packages\transformers\utils\import_utils.py", line 1076, in _get_module
  File "C:\Users\Sumiran Grover\anaconda3\lib\importlib\__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1004, in _find_and_load_unlocked
ModuleNotFoundError: No module named 'transformers.models.xlm_prophetnet.configuration_xlm_prophetnet'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Sumiran Grover\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Sumiran Grover\AppData\Local\Temp\ipykernel_47172\2416997931.py", line

In [35]:
class AdvancedResearchSummarizer:
    def extractive_summary(self, text, max_sentences=3):
        # Enhance sentence selection criteria
        sentences = list(self.nlp(text).sents)
        
        # Score sentences based on multiple factors
        def sentence_score(sentence):
            # Linguistic features
            noun_chunks = len(list(sentence.noun_chunks))
            named_entities = len(list(sentence.ents))
            
            # TF-IDF scoring
            tfidf_score = self.vectorizer.transform([sentence.text]).sum()
            
            # Combine scoring mechanisms
            return (
                noun_chunks * 0.3 + 
                named_entities * 0.2 + 
                tfidf_score * 0.5
            )
        
        # Rank sentences
        scored_sentences = sorted(
            sentences, 
            key=sentence_score, 
            reverse=True
        )
        
        # Select top sentences maintaining original order
        selected_sentences = sorted(
            scored_sentences[:max_sentences], 
            key=lambda s: text.index(s.text)
        )
        
        return ' '.join([str(sent) for sent in selected_sentences])
    
    
    


In [37]:
# Main execution
pdf_path = "A_Comprehensive_Review_of_Unimodal_and_Multimodal_Fingerprint_Biometric_Authentication_Systems_Fusion_Attacks_and_Template_Protection.pdf"
summaries = process_research_paper(pdf_path)
summaries = process_research_paper(pdf_path)

Conclusion Summary:
As the benefits of fingerprint biometric systems are being discussed, several application scenarios illustrating the algorithms used to construct fin gerprint biometric systems are emphasized. We found that although some devices that use dynamic biometrics need to improve their verification accuracy, most of the current solutions have privacy and security flaws. Of all the levels of data fusion, score level fusion continues to be the most beneficial because it is simple to identify and merge the matching scores. Multimodal biometric systems are to be improved by incorporating additional sensors, improving matching algorithms, handling noise errors, and analyzing data. S. Dargan and M. Kumar, A comprehensive survey on the biometric recognition systems based on physiological and behavioral modalities, Expert Syst.
--------------------------------------------------
Introduction Summary:
C. Kant and S. Chaudhary, A multimodal biometric system based on finger knuckle pri

In [36]:
def advanced_summarization(sections):
    summarizer = AdvancedResearchSummarizer()
    refined_summaries = {}
    
    # Section-specific processing
    section_priorities = {
        'Abstract': 5,   # Most important
        'Conclusion': 4, # High priority
        'Introduction': 3,
        'Discussion': 3,
        'Results': 2,
        'Methods': 2,
        'Title': 1       # Least priority
    }
    
    for section, content in sections.items():
        # Dynamic sentence count based on section importance
        max_sentences = section_priorities.get(section, 2)
        refined_summaries[section] = summarizer.extractive_summary(
            content, 
            max_sentences=max_sentences
        )
    
    return refined_summaries

In [41]:
import os
import PyPDF2
import spacy
import nltk
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine

In [45]:

class ResearchPaperProcessor:
    def __init__(self):
        # Download essential NLP resources
        nltk.download('punkt', quiet=True)
        
        # Load scientific NLP model
        self.nlp = spacy.load('en_core_web_sm')

    def extract_text_from_pdf(self, pdf_path):
        """Extract raw text from PDF"""
        try:
            with open(pdf_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                text = " ".join(page.extract_text() for page in reader.pages)
            return text
        except Exception as e:
            print(f"Error extracting PDF text: {e}")
            return ""

    def preprocess_text(self, text):
        """Clean and normalize text"""
        text = text.lower()
        doc = self.nlp(text)
        cleaned_tokens = [
            token.text for token in doc 
            if not token.is_stop and not token.is_punct
        ]
        return ' '.join(cleaned_tokens)

    def sentence_segmentation(self, text):
        """Segment text into sentences"""
        return nltk.sent_tokenize(text)

    def generate_extractive_summary(self, text, max_sentences=5):
        """
        Generate extractive summary using TF-IDF
        
        Handles matrix operations more robustly
        """
        sentences = self.sentence_segmentation(text)
        
        if len(sentences) <= max_sentences:
            return sentences
        
        # TF-IDF Vectorization
        vectorizer = TfidfVectorizer()
        try:
            tfidf_matrix = vectorizer.fit_transform(sentences)
            
            # Convert matrix to dense numpy array for calculations
            dense_matrix = tfidf_matrix.toarray()
            
            # Calculate centroid
            centroid = np.mean(dense_matrix, axis=0)
            
            # Calculate sentence scores
            sentence_scores = [
                1 - cosine(sent_vec, centroid) 
                for sent_vec in dense_matrix
            ]
            
            # Select top sentences
            top_sentence_indices = sorted(
                range(len(sentence_scores)), 
                key=lambda i: sentence_scores[i], 
                reverse=True
            )[:max_sentences]
            
            return [sentences[i] for i in top_sentence_indices]
        
        except Exception as e:
            print(f"Error in summary generation: {e}")
            # Fallback: return first max_sentences
            return sentences[:max_sentences]

    def morphological_analysis(self, text):
        """Perform morphological analysis"""
        doc = self.nlp(text)
        
        morphology_insights = {
            'pos_distribution': {},
            'named_entities': [],
            'dependency_relations': {}
        }
        
        # Part of Speech Distribution
        for token in doc:
            pos = token.pos_
            morphology_insights['pos_distribution'][pos] = \
                morphology_insights['pos_distribution'].get(pos, 0) + 1
        
        # Named Entities
        morphology_insights['named_entities'] = [
            (ent.text, ent.label_) for ent in doc.ents
        ]
        
        # Dependency Relations
        for token in doc:
            if token.dep_ not in morphology_insights['dependency_relations']:
                morphology_insights['dependency_relations'][token.dep_] = []
            morphology_insights['dependency_relations'][token.dep_].append(token.text)
        
        return morphology_insights

    def advanced_summary(self, text):
        """Generate comprehensive summary"""
        preprocessed_text = self.preprocess_text(text)
        morphological_data = self.morphological_analysis(preprocessed_text)
        extractive_summary = self.generate_extractive_summary(preprocessed_text)
        
        return {
            'preprocessed_text': preprocessed_text,
            'morphological_insights': morphological_data,
            'extractive_summary': extractive_summary
        }

    def visualize_semantic_graph(self, text):
        """Create semantic relationship graph"""
        doc = self.nlp(text)
        G = nx.DiGraph()
        
        for sent in doc.sents:
            for token in sent:
                if token.dep_ in ['nsubj', 'dobj', 'pobj']:
                    G.add_node(token.text)
                    
                    # Connect related tokens
                    for child in token.children:
                        G.add_edge(token.text, child.text)
        
        plt.figure(figsize=(12, 8))
        pos = nx.spring_layout(G)
        nx.draw_networkx_nodes(G, pos, node_color='lightblue')
        nx.draw_networkx_edges(G, pos)
        nx.draw_networkx_labels(G, pos)
        plt.title("Semantic Relationship Graph")
        plt.axis('off')
        plt.tight_layout()
        plt.savefig('semantic_graph.png')
        
        return G

def process_research_paper(pdf_path):
    """Process research paper and generate insights"""
    processor = ResearchPaperProcessor()
    
    # Extract text
    raw_text = processor.extract_text_from_pdf(pdf_path)
    
    # Generate comprehensive analysis
    summary = processor.advanced_summary(raw_text)
    
    # Visualize semantic relationships
    semantic_graph = processor.visualize_semantic_graph(raw_text)
    
    return summary, semantic_graph
